In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mbti_dataset.csv')
#print(df.columns)
#print(df.head())
# MBTI 레이블을 개별 차원으로 분할
def split_mbti_label(label):
    dimensions = ['e', 'i'], ['s', 'n'], ['t', 'f'], ['j', 'p']
    result = []
    for options in dimensions:
        if label[dimensions.index(options)] == options[0]:
            result.append(0)  # 첫 번째 옵션 (e, s, t, j)
        else:
            result.append(1)  # 두 번째 옵션 (i, n, f, p)
    return result

# apply 함수를 사용하여 label을 분할하고 새로운 열을 생성
df[['label_EI', 'label_SN', 'label_TF', 'label_JP']] = df['label'].apply(split_mbti_label).apply(pd.Series)
print(df.head())

                                                text label  label_EI  \
0         공방 가서 맞추는 건 싫고 금으로 맞추는 거 아님[SEP]싫어요 ㅎ[SEP]  istj         1   
1                  원하시는 일상 공유가 어떤 건지 [SEP]궁금해요 [SEP]  intp         1   
2                  저는 무조건 ..![SEP]말해야 된다고 생각해요 [SEP]  enfp         0   
3  자기 정도만 쓰는 거 같아요.[SEP] 애인을 지칭할 때 '너'라고 하기에는 너무 ...  istj         1   
4  직장 상사 스타일이라고 생각하심 돼요 [SEP]제가 느끼기에는 f 부분이 너무 적어...  estp         0   

   label_SN  label_TF  label_JP  
0         0         0         0  
1         1         0         1  
2         1         1         1  
3         0         0         0  
4         0         0         1  


In [ ]:
mbti_counts = df['label'].value_counts()

# mbti 별 data 갯수 체크
mbti_counts_df = mbti_counts.reset_index()
mbti_counts_df.columns = ['text', 'label']

print(mbti_counts_df)


    text  label
0   infp   4140
1   entp   3006
2   intp   2982
3   intj   2863
4   infj   2639
5   istp   2613
6   enfp   2362
7   isfp   2333
8   enfj   1547
9   entj   1475
10  isfj   1332
11  estp   1272
12  estj   1194
13  istj   1104
14  esfj   1102
15  esfp   1062


In [ ]:
pip install wordcloud matplotlib pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/mbti_dataset.csv')

# MBTI 유형 목록
mbti_types = [
    'istj', 'isfj', 'infj', 'intj',
    'istp', 'isfp', 'infp', 'intp',
    'estp', 'esfp', 'enfp', 'entp',
    'estj', 'esfj', 'enfj', 'entj'
]

# 워드 클라우드 생성 함수
def generate_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# 각 MBTI 유형별로 워드 클라우드 생성
for mbti in mbti_types:
    # 해당 유형의 텍스트 데이터 추출
    mbti_data = data[data['label'] == mbti]
    text = ' '.join(mbti_data['text'].values)

    # 워드 클라우드 생성 및 시각화
    generate_wordcloud(text, title=f'Word Cloud for {mbti.upper()}')


ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
longest_string = df['text'].astype(str).map(len).max()
longest_string_row = df[df['text'].astype(str).map(len) == longest_string]
# longest_string_row
df.iloc[3285]['text']

'했습니다.[SEP] 오늘 행복하게 보낼 수 있으려나[SEP]'

In [ ]:
from collections import defaultdict
from sklearn.model_selection import train_test_split

# 텍스트와 새로운 라벨 열들로 데이터 준비
data = [{
    'text': row['text'],
    'label_EI': row['label_EI'],
    'label_SN': row['label_SN'],
    'label_TF': row['label_TF'],
    'label_JP': row['label_JP']
} for row in df[['text', 'label_EI', 'label_SN', 'label_TF', 'label_JP']].to_dict('records')]

# 데이터셋을 train, val, test 세트로 분할
train_data, tmp_test_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(tmp_test_data, test_size=0.5, random_state=42)

# 데이터 크기 출력
print(f"Train data size: {len(train_data)}")
print(f"Validation data size: {len(val_data)}")
print(f"Test data size: {len(test_data)}")

Train data size: 26420
Validation data size: 3303
Test data size: 3303


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW
from tqdm import tqdm

# 1. 데이터 전처리
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

class MBTIDataset(Dataset):
    def __init__(self, data, max_len):
        self.data = data
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data[index]['text']
        label_EI = self.data[index]['label_EI']
        label_SN = self.data[index]['label_SN']
        label_TF = self.data[index]['label_TF']
        label_JP = self.data[index]['label_JP']
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label_EI': torch.tensor(label_EI),
            'label_SN': torch.tensor(label_SN),
            'label_TF': torch.tensor(label_TF),
            'label_JP': torch.tensor(label_JP)
        }

# 2. 데이터로더 준비
def clean_data(data):
    return [item for item in data if isinstance(item['text'], str) and item['text'].strip()]

train_dataset = MBTIDataset(clean_data(train_data), max_len=128)
val_dataset = MBTIDataset(clean_data(val_data), max_len=128)
test_dataset = MBTIDataset(clean_data(test_data), max_len=128)

batch_size = 384

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

In [ ]:
import os
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW
from tqdm import tqdm

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# 1. BERT 모델 및 분류기 준비
model = BertModel.from_pretrained("kykim/bert-kor-base").to(device)

# BERT 모델 파라미터를 고정 (freeze)
for param in model.parameters():
    param.requires_grad = False

# 분류기 정의
class MBTIMultiLabelClassifier(torch.nn.Module):
    def __init__(self, bert_model):
        super(MBTIMultiLabelClassifier, self).__init__()
        self.bert = bert_model
        self.classifier_EI = torch.nn.Linear(self.bert.config.hidden_size, 2)
        self.classifier_SN = torch.nn.Linear(self.bert.config.hidden_size, 2)
        self.classifier_TF = torch.nn.Linear(self.bert.config.hidden_size, 2)
        self.classifier_JP = torch.nn.Linear(self.bert.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits_EI = self.classifier_EI(pooled_output)
        logits_SN = self.classifier_SN(pooled_output)
        logits_TF = self.classifier_TF(pooled_output)
        logits_JP = self.classifier_JP(pooled_output)
        return logits_EI, logits_SN, logits_TF, logits_JP

classifier = MBTIMultiLabelClassifier(model).to(device)

# 2. 손실 함수와 옵티마이저 정의
loss_fn = torch.nn.CrossEntropyLoss().to(device)
optimizer = AdamW(classifier.parameters(), lr=5e-3) # 0.005

# 3. 학습 loop
epochs = 3  # DataSet을 1회 온전하게 Weight Update하는데 사용.
for epoch in range(epochs):
    model.eval()  # BERT 모델을 평가 모드로 설정 (학습 비활성화)
    classifier.train()  # 분류기는 학습 모드로 설정
    loop = tqdm(train_loader)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label_EI = batch['label_EI'].to(device)
        label_SN = batch['label_SN'].to(device)
        label_TF = batch['label_TF'].to(device)
        label_JP = batch['label_JP'].to(device)

        with torch.no_grad():  # BERT 모델의 기울기 계산 비활성화
            outputs = model(input_ids, attention_mask=attention_mask)

        logits_EI, logits_SN, logits_TF, logits_JP = classifier(input_ids, attention_mask)
        loss_EI = loss_fn(logits_EI, label_EI)
        loss_SN = loss_fn(logits_SN, label_SN)
        loss_TF = loss_fn(logits_TF, label_TF)
        loss_JP = loss_fn(logits_JP, label_JP)
        loss = loss_EI + loss_SN + loss_TF + loss_JP
        loss.backward()
        optimizer.step()
        loop.set_postfix(loss=loss.item())

    classifier.eval()  # 분류기를 평가 모드로 설정
    val_loss = 0
    val_acc_EI = 0
    val_acc_SN = 0
    val_acc_TF = 0
    val_acc_JP = 0
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label_EI = batch['label_EI'].to(device)
        label_SN = batch['label_SN'].to(device)
        label_TF = batch['label_TF'].to(device)
        label_JP = batch['label_JP'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits_EI, logits_SN, logits_TF, logits_JP = classifier(input_ids, attention_mask)

            loss_EI = loss_fn(logits_EI, label_EI)
            loss_SN = loss_fn(logits_SN, label_SN)
            loss_TF = loss_fn(logits_TF, label_TF)
            loss_JP = loss_fn(logits_JP, label_JP)
            loss = loss_EI + loss_SN + loss_TF + loss_JP

            val_loss += loss.item()
            preds_EI = torch.argmax(logits_EI, dim=1)
            preds_SN = torch.argmax(logits_SN, dim=1)
            preds_TF = torch.argmax(logits_TF, dim=1)
            preds_JP = torch.argmax(logits_JP, dim=1)
            val_acc_EI += (preds_EI == label_EI).sum().item()
            val_acc_SN += (preds_SN == label_SN).sum().item()
            val_acc_TF += (preds_TF == label_TF).sum().item()
            val_acc_JP += (preds_JP == label_JP).sum().item()

    val_loss /= len(val_loader)
    val_acc_EI /= len(val_dataset)
    val_acc_SN /= len(val_dataset)
    val_acc_TF /= len(val_dataset)
    val_acc_JP /= len(val_dataset)
    val_acc = (val_acc_EI + val_acc_SN + val_acc_TF + val_acc_JP) / 4
    print(f'Epoch {epoch+1} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')
    print(f'Label Val Acc - EI: {val_acc_EI:.4f}, SN: {val_acc_SN:.4f}, TF: {val_acc_TF:.4f}, JP: {val_acc_JP:.4f}')

# 6. 모델 평가 및 저장
model.eval()
classifier.eval()
test_acc_EI = 0
test_acc_SN = 0
test_acc_TF = 0
test_acc_JP = 0
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    label_EI = batch['label_EI'].to(device)
    label_SN = batch['label_SN'].to(device)
    label_TF = batch['label_TF'].to(device)
    label_JP = batch['label_JP'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits_EI, logits_SN, logits_TF, logits_JP = classifier(input_ids, attention_mask)
        preds_EI = torch.argmax(logits_EI, dim=1)
        preds_SN = torch.argmax(logits_SN, dim=1)
        preds_TF = torch.argmax(logits_TF, dim=1)
        preds_JP = torch.argmax(logits_JP, dim=1)
        test_acc_EI += (preds_EI == label_EI).sum().item()
        test_acc_SN += (preds_SN == label_SN).sum().item()
        test_acc_TF += (preds_TF == label_TF).sum().item()
        test_acc_JP += (preds_JP == label_JP).sum().item()
test_acc_EI /= len(test_dataset)
test_acc_SN /= len(test_dataset)
test_acc_TF /= len(test_dataset)
test_acc_JP /= len(test_dataset)
test_acc = (test_acc_EI + test_acc_SN + test_acc_TF + test_acc_JP) / 4
print(f'Test Acc: {test_acc:.4f} | EI: {val_acc_EI:.4f} | SN: {test_acc_SN:.4f} | TF: {val_acc_TF:.4f} | JP: {val_acc_JP:.4f}')

# 모델 저장
classifier.cpu()
classifier_weight_path = '/content/drive/MyDrive/Colab Notebooks/kobert-mbti-classifier-softmax.pt'
torch.save(classifier.state_dict(), classifier_weight_path)

optimizer_state_path = '/content/drive/MyDrive/Colab Notebooks/kobert-mbti-optimizer-softmax.pt'
training_state_path = '/content/drive/MyDrive/Colab Notebooks/kobert-mbti-training-state-softmax.pt'

training_state = {
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'classifier_state_dict': classifier.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}

torch.save(optimizer.state_dict(), optimizer_state_path)
torch.save(training_state, training_state_path)


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# GPU 사용 여부
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 토크나이저 및 모델 로드
classifier_weight_path = '/content/drive/MyDrive/Colab Notebooks/kobert-mbti-classifier-softmax.pt'
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')
model = BertModel.from_pretrained('kykim/bert-kor-base').to(device)

classifier = MBTIMultiLabelClassifier(model).to(device)

# 가중치 로드
classifier.load_state_dict(torch.load(classifier_weight_path))
model.eval()
classifier.eval()
model.to(device)

# 입력된 문장을 기반으로 MBTI 예측 함수
def predict_mbti(text):
    # 입력된 문장을 토큰화
    encoding = tokenizer.encode_plus(
        text,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        # BERT 모델을 사용하여 특징 추출
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        # 분류기를 통해 각 MBTI 차원 예측
        logits_EI, logits_SN, logits_TF, logits_JP = classifier(input_ids, attention_mask)

    # 각 차원에 대해 가장 높은 확률을 가진 클래스 선택
    pred_EI = torch.argmax(logits_EI, dim=1).item()
    pred_SN = torch.argmax(logits_SN, dim=1).item()
    pred_TF = torch.argmax(logits_TF, dim=1).item()
    pred_JP = torch.argmax(logits_JP, dim=1).item()

    # 예측된 클래스를 MBTI 유형으로 변환
    mbti = ''
    mbti += 'E' if pred_EI == 0 else 'I'
    mbti += 'S' if pred_SN == 0 else 'N'
    mbti += 'T' if pred_TF == 0 else 'F'
    mbti += 'J' if pred_JP == 0 else 'P'

    return mbti

# 문장 입력받아 예측
input_text = "현재 너무 화나서 아이스크림을 사러가고 있어"
predicted_mbti = predict_mbti(input_text)
print(f"Predicted MBTI: {predicted_mbti}")

torch.cuda.empty_cache()
if torch.cuda.is_available():
  torch.cuda.empty_cache()
  torch.cuda.ipc_collect()

Predicted MBTI: ENFP


In [ ]:
## ablation study ##
# 1. BERT 모델의 특정 레이어 제거
# 2. 분류기 헤드 변경(최종)
# 3. 특정 입력 특징 제거 stop words?
# 4. 파라미터 민감도 분석(learning rate, batch size 조정해보기)

# 1. 특정 레이어 제거
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, AdamW
from tqdm import tqdm
from transformers import BertModel
import torch.nn as nn

# 데이터 전처리 및 데이터 로더 설정
tokenizer = BertTokenizer.from_pretrained('kykim/bert-kor-base')

# 데이터 로더 준비
def clean_data(data):
    return [item for item in data if isinstance(item['text'], str) and item['text'].strip()]

train_dataset = MBTIDataset(clean_data(train_data), max_len=128)
val_dataset = MBTIDataset(clean_data(val_data), max_len=128)
test_dataset = MBTIDataset(clean_data(test_data), max_len=128)

batch_size = 384

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# GPU 사용 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

class ModifiedBertModel(BertModel):
    def __init__(self, config, ablation_layer):
        super().__init__(config)
        self.ablation_layer = ablation_layer

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=None, output_hidden_states=None, return_dict=None):
        outputs = super().forward(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=True,  # 모든 레이어의 히든 상태를 출력하도록 설정
            return_dict=return_dict
        )

        hidden_states = outputs.hidden_states  # 모든 레이어의 히든 상태

        # 특정 레이어를 제거하고 나머지 레이어의 hidden state를 concat
        modified_hidden_states = torch.cat([hidden_state for i, hidden_state in enumerate(hidden_states) if i != self.ablation_layer], dim=-1)

        return modified_hidden_states

train_data = clean_data(train_data)
val_data = clean_data(val_data)
test_data = clean_data(test_data)

# BERT 모델 및 분류기 준비
ABLATION_LAYER = 8  # 제거할 BERT 레이어

model = ModifiedBertModel.from_pretrained("kykim/bert-kor-base", ablation_layer=ABLATION_LAYER).to(device)

# 분류기 정의
class MBTIMultiLabelClassifier(nn.Module):
    def __init__(self, bert_model):
        super(MBTIMultiLabelClassifier, self).__init__()
        self.bert = bert_model
        hidden_size = self.bert.config.hidden_size * (self.bert.config.num_hidden_layers)
        self.classifier_EI = nn.Linear(hidden_size, 2)
        self.classifier_SN = nn.Linear(hidden_size, 2)
        self.classifier_TF = nn.Linear(hidden_size, 2)
        self.classifier_JP = nn.Linear(hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        modified_hidden_states = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = modified_hidden_states.mean(dim=1)  # 평균 풀링(입력 시퀀스의 모든 토큰에 대한 히든 상태의 평균을 계산하여 하나의 벡터로 만듬)
        logits_EI = self.classifier_EI(pooled_output)
        logits_SN = self.classifier_SN(pooled_output)
        logits_TF = self.classifier_TF(pooled_output)
        logits_JP = self.classifier_JP(pooled_output)
        return logits_EI, logits_SN, logits_TF, logits_JP

classifier = MBTIMultiLabelClassifier(model).to(device)

# 손실 함수와 옵티마이저 정의
loss_fn = torch.nn.CrossEntropyLoss().to(device)
optimizer = AdamW(classifier.parameters(), lr=5e-3)

# 학습 및 평가 함수
def train_and_evaluate(model, classifier, train_loader, val_loader, test_loader, epochs=3):
    loss_fn = nn.CrossEntropyLoss().to(device)
    optimizer = AdamW(classifier.parameters(), lr=5e-3)  # 0.005

    for epoch in range(epochs):
        model.eval()  # BERT 모델을 평가 모드로 설정 (학습 비활성화)
        classifier.train()  # 분류기는 학습 모드로 설정
        loop = tqdm(train_loader)
        for batch in loop:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label_EI = batch['label_EI'].to(device)
            label_SN = batch['label_SN'].to(device)
            label_TF = batch['label_TF'].to(device)
            label_JP = batch['label_JP'].to(device)

            with torch.no_grad():  # BERT 모델의 기울기 계산 비활성화
                modified_hidden_states = model(input_ids, attention_mask=attention_mask)

            pooled_output = modified_hidden_states.mean(dim=1)  # 평균 풀링
            logits_EI, logits_SN, logits_TF, logits_JP = classifier(input_ids, attention_mask)
            loss_EI = loss_fn(logits_EI, label_EI)
            loss_SN = loss_fn(logits_SN, label_SN)
            loss_TF = loss_fn(logits_TF, label_TF)
            loss_JP = loss_fn(logits_JP, label_JP)
            loss = loss_EI + loss_SN + loss_TF + loss_JP
            loss.backward()
            optimizer.step()
            loop.set_postfix(loss=loss.item())

        classifier.eval()  # 분류기를 평가 모드로 설정
        val_loss = 0
        val_acc_EI = 0
        val_acc_SN = 0
        val_acc_TF = 0
        val_acc_JP = 0
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label_EI = batch['label_EI'].to(device)
            label_SN = batch['label_SN'].to(device)
            label_TF = batch['label_TF'].to(device)
            label_JP = batch['label_JP'].to(device)

            with torch.no_grad():
                modified_hidden_states = model(input_ids, attention_mask=attention_mask)
                pooled_output = modified_hidden_states.mean(dim=1)
                logits_EI, logits_SN, logits_TF, logits_JP = classifier(input_ids, attention_mask)

                loss_EI = loss_fn(logits_EI, label_EI)
                loss_SN = loss_fn(logits_SN, label_SN)
                loss_TF = loss_fn(logits_TF, label_TF)
                loss_JP = loss_fn(logits_JP, label_JP)
                loss = loss_EI + loss_SN + loss_TF + loss_JP

                val_loss += loss.item()
                preds_EI = torch.argmax(logits_EI, dim=1)
                preds_SN = torch.argmax(logits_SN, dim=1)
                preds_TF = torch.argmax(logits_TF, dim=1)
                preds_JP = torch.argmax(logits_JP, dim=1)
                val_acc_EI += (preds_EI == label_EI).sum().item()
                val_acc_SN += (preds_SN == label_SN).sum().item()
                val_acc_TF += (preds_TF == label_TF).sum().item()
                val_acc_JP += (preds_JP == label_JP).sum().item()

        val_loss /= len(val_loader)
        val_acc_EI /= len(val_dataset)
        val_acc_SN /= len(val_dataset)
        val_acc_TF /= len(val_dataset)
        val_acc_JP /= len(val_dataset)
        val_acc = (val_acc_EI + val_acc_SN + val_acc_TF + val_acc_JP) / 4
        print(f'Epoch {epoch+1} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')
        print(f'Label Val Acc - EI: {val_acc_EI:.4f}, SN: {val_acc_SN:.4f}, TF: {val_acc_TF:.4f}, JP: {val_acc_JP:.4f}')

# 모델 학습 및 평가
num_epochs = 3
train_and_evaluate(model, classifier, train_loader, val_loader, test_loader, epochs=num_epochs)


Device: cpu


100%|██████████| 69/69 [55:20<00:00, 48.12s/it, loss=6.05]


Epoch 1 | Val Loss: 5.6017 | Val Acc: 0.5325
Label Val Acc - EI: 0.6210, SN: 0.6152, TF: 0.5005, JP: 0.3936


100%|██████████| 69/69 [55:19<00:00, 48.11s/it, loss=3.7]


Epoch 2 | Val Loss: 2.9075 | Val Acc: 0.4723
Label Val Acc - EI: 0.6140, SN: 0.3585, TF: 0.5244, JP: 0.3924


100%|██████████| 69/69 [55:01<00:00, 47.85s/it, loss=8.78]


Epoch 3 | Val Loss: 6.5845 | Val Acc: 0.5911
Label Val Acc - EI: 0.6210, SN: 0.6415, TF: 0.4941, JP: 0.6076


In [ ]:
# 예측 함수
def predict(text, model, classifier, tokenizer, device):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        modified_hidden_states, pooled_output = outputs
        logits_EI, logits_SN, logits_TF, logits_JP = classifier(pooled_output)
        preds_EI = torch.argmax(logits_EI, dim=1).item()
        preds_SN = torch.argmax(logits_SN, dim=1).item()
        preds_TF = torch.argmax(logits_TF, dim=1).item()
        preds_JP = torch.argmax(logits_JP, dim=1).item()

    mbti_type = ""
    if preds_EI == 0:
        mbti_type += "E"
    else:
        mbti_type += "I"
    if preds_SN == 0:
        mbti_type += "S"
    else:
        mbti_type += "N"
    if preds_TF == 0:
        mbti_type += "T"
    else:
        mbti_type += "F"
    if preds_JP == 0:
        mbti_type += "J"
    else:
        mbti_type += "P"

    return mbti_type

# 예측 코드 실행
text = "저는 책 읽는 것을 좋아하고 혼자 있는 시간을 즐기는 편입니다."
mbti_type = predict(text, model, classifier, tokenizer, device)
print(f"예측된 MBTI 유형: {mbti_type}")

In [ ]:
torch.cuda.empty_cache()
if torch.cuda.is_available():
  torch.cuda.empty_cache()
  torch.cuda.ipc_collect()